In [ ]:
import pandas as pd
import numpy as np
import bson
import matplotlib.pyplot as plt
import cv2 #opencv library 
from skimage import data, io, filters, feature #essential when extracting features


from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny,hog,corner_harris

In [ ]:
path = '../input/'
!ls "$path"

#### Read Files

In [ ]:
categories = pd.read_csv('{}{}'.format(path,'category_names.csv'),
                         index_col=0)

In [ ]:
categories.head()

#### Read bson file and convert to pandas DataFrame

In [ ]:
with open('{}{}'.format(path,'train_example.bson'),'rb') as b:
    df = pd.DataFrame(bson.decode_all(b.read()))
#read and convert to opencv image
df['imgs'] = df['imgs'].apply(lambda rec: cv2.imdecode(
    np.fromstring(rec[0]['picture'], np.uint8), cv2.IMREAD_COLOR))
#change index to catergory id
df.set_index('category_id',inplace=True)
# Combine images and categries
df[categories.columns.tolist()] = categories.loc[df.index]

###### Run through filter

In [ ]:
def get_edges(img):
    #convert to grayscale
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = filters.prewitt_h(gray)
    return edges
df['prewitt_img'] = df['imgs'].apply(get_edges)

### Conrner detection

In [ ]:
def get_corners(img):
    #convert to grayscale
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    corners = feature.corner_fast(gray)
    return corners
df['corners'] = df['imgs'].apply(get_corners)

In [ ]:
df['corners'].head()

In [ ]:
fig,axs  = plt.subplots(4,4,figsize=(8,8))
title = df['category_level1'].str.split('-').str[0].str.strip()
# title += ','
# title += df['category_level2'].str.split('-').str[0].str.strip()
#specify which block of images to display
n=50
title = title.tolist()

axs = axs.flatten()

for i,ax in enumerate(axs):
    ax.imshow(df.iloc[i+n,1][:,:,1],cmap='gray')
    #convert the edges matrix to image array
    ax.contour((np.interp(df.iloc[i+n,-2],
                      [df.iloc[i+n,-2].min(),df.iloc[i+n,-1].max()], #old range
                      [0,255] #new range
                        )>10).astype(np.uint8),colors='b',lw=.5)
    ax.contour((df.iloc[i+n,-1]).astype(np.uint8),
               colors='r',lw=3.)
    ax.set_title(title[i+n])
    #remove frame and ticks
    ax.axis('off')

plt.tight_layout()
plt.suptitle('Filter+corner')
 #make room for main title
fig.subplots_adjust(top=0.90)
#create and show legend
proxy = [plt.Rectangle((0,0),1,1,fc = pc) for pc in ['gray','blue','red']]
axs[0].legend(proxy, ['Grayscale image','Prewitt_h Filter','Corner detection'],
           bbox_to_anchor=(0.1,.9), loc="upper right",frameon=False);


In [ ]:

image = df[-10][...,1]
#pass the image through a canny edge detecter
edges = canny(image, 2, 1, 25)
#apply hough transform
lines = probabilistic_hough_line(edges, threshold=20, line_length=10,
                                 line_gap=3)

# Generating figure 2
fig, axes = plt.subplots(1, 3)
axs = axes.flatten()

axs[0].imshow(image, cmap=cm.gray)
axs[0].set_title('Input image')

axs[1].imshow(edges, cmap=cm.gray)
axs[1].set_title('Canny edges')

axs[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()
    a.set_adjustable('box-forced')

plt.tight_layout()
plt.show()